In [16]:
library(edgeR)
library(magrittr)
library(ggplot2)
library(DESeq2)

Loading required package: limma

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tap

# Automatise

# On cell-state level

## For LV

In [1]:
x=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_ALL_CD4CD8NK_PSEUDOBULK_LV.csv")
genes_tofilter=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_ALL_CD4CD8NK_PSEUDOBULK_FILTERING_LV.csv")

# Only needed for the column cell_state
CELLTYPE_STATE <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_CD4CD8NK_TRANSLATION_TABLE.csv")
colnames(CELLTYPE_STATE) <- c("cell_states", 'cell_type')

# Removes patients with only few (<10) nuclei numbers (as they might bias the analysis)
CELLTYPE_FILTER <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_ABSOLUTE_CELLTYPE_NUMBER_LV.csv")


In [13]:
PATIENTS_TOKEEP <- CELLTYPE_FILTER[CELLTYPE_FILTER$celltype_classified=="Lymphoid",2:43]>=30

In [14]:
PATIENTS_TOKEEP <- names(which(PATIENTS_TOKEEP[1,]))

In [7]:
rownames(x) <- x$X
x <- x[,-1]
l <- strsplit(colnames(x), "__")

In [8]:
meta.data <- t(as.data.frame(l))
rownames(meta.data) <- colnames(x)

meta.data <- as.data.frame(meta.data)
colnames(meta.data)  <-  c("Condition_custom", "cell_state", 'Patient')

In [10]:
GENOTYPES <- unique(meta.data[,"Condition_custom"][meta.data[,"Condition_custom"] != "Control"])
GENOTYPES

[1] Myocarditis      Pims             Post.Covid.19    Post.Vaccination
Levels: Control Myocarditis Pims Post.Covid.19 Post.Vaccination

In [11]:
CELL_LEVEL <- "CELLSTATES"
REGION <- "LV"

In [ ]:
i <- 1

for(GENOTYPE in as.character(GENOTYPES)){
    message("\n###START: ", GENOTYPE," #####\n")
    
    for(CELL_STATE in as.character(unique(meta.data$cell_state))){
    #for(CELL_STATE in c("EC7")){
        
        CONTROL_COLUMN <- paste("control_",CELL_STATE, sep="")
        GENOTYPE_COLUMN <- paste(GENOTYPE,"_", CELL_STATE, sep="")
        CELLTYPE <- as.character(CELLTYPE_STATE[which(CELLTYPE_STATE$cell_states==CELL_STATE),"cell_type"])
        
        
        x_sub <- x[,which(meta.data[,"Condition_custom"] %in% c("Control", GENOTYPE))]
        meta.data_sub <- meta.data[which(meta.data[,"Condition_custom"] %in% c("Control", GENOTYPE)),]
        
        x_sub <- x_sub[,which(meta.data_sub$cell_state %in% c(CELL_STATE))]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$cell_state %in% c(CELL_STATE)),]
        
        ##### remove first line (cell_type column)
        #CELLTYPE_FILTER_SUB <- CELLTYPE_FILTER
        #PATIENTS_TOKEEP <- CELLTYPE_FILTER_SUB[which(CELLTYPE_FILTER_SUB$cell_state==CELL_STATE),-1]
        #PATIENTS_TOKEEP <- colnames(PATIENTS_TOKEEP)[which(as.logical(PATIENTS_TOKEEP))]
        #####
        
        # Subset only for patients which fulfill condition
        x_sub <- x_sub[,which(meta.data_sub$Patient %in% PATIENTS_TOKEEP)]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$Patient %in% PATIENTS_TOKEEP),]
        
        meta.data_sub[,"Condition_custom"] <- factor(as.character(meta.data_sub[,"Condition_custom"]))
        #meta.data_sub[,"Condition_custom"] = droplevels(meta.data_sub[,"Condition_custom"]) ## Drop factor levels
   
        
        # The test is only run, if 2 conditions have at least 3 patients
        if (length(unique(meta.data_sub[,"Condition_custom"]))==2 & 
            all(table(as.character(meta.data_sub[,"Condition_custom"]))>=3))
           {
             # Prepare DGEList object
            meta.data_sub[,"Condition_custom"] <- as.factor(as.character(meta.data_sub[,"Condition_custom"]))
            dge <- DGEList(counts=x_sub, group=meta.data_sub[,"Condition_custom"])
            
            # Filter genes based on expression, the qlf Object will be filtered to exclude them from FDR calculation
            genes_tofilter_sub_GENOTYPE <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% GENOTYPE &
                                                                 gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE &
                                                                 gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]

            genes_tofilter_sub_CONTROL <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% 'Control' &
                                                                gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE &
                                                                 gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]
            
            genes_tofilter_sub <- data.frame("X"=genes_tofilter[,"X"],
                                             CONTROL_COLUMN=apply(genes_tofilter_sub_CONTROL, 1, mean),
                                             GENOTYPE_COLUMN=apply(genes_tofilter_sub_GENOTYPE, 1, mean))
            colnames(genes_tofilter_sub) <- c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)
            
            keep <- which(genes_tofilter_sub[,CONTROL_COLUMN] > 0.0125 |
                          genes_tofilter_sub[,GENOTYPE_COLUMN] > 0.0125)
            
            #keep <- genes_tofilter[,CONTROL_COLUMN]>0.0125 | ##### change & to | ##### 
            #        genes_tofilter[,GENOTYPE_COLUMN]>0.0125
            #dge <- dge[keep, , keep.lib.sizes=FALSE]
    
            # PP, model matrix, https://www.nature.com/articles/nmeth.4612 (edgeRQLFDetRate)
            #dge <- calcNormFactors(dge)
            #cdr <- scale(colMeans(x_sub > 0))
            #design <- model.matrix(~ cdr + meta.data_sub[,"Condition_custom"])
            design <- model.matrix(~ meta.data_sub[,"Condition_custom"])
    
            # 
            dge <- estimateDisp(dge, design = design)
            fit <- glmQLFit(dge, design = design)
            qlf <- glmQLFTest(fit)
    
            # For all
            tt <- topTags(qlf, n = Inf)
            
            # Only for "expressed/detected" genes
            tt_filtered <- topTags(qlf[keep,], n = Inf)
            
            tt$table[,"Gene"] <- as.character(rownames(tt$table))
            tt_filtered$table[,"Gene"] <- as.character(rownames(tt_filtered$table))
            tt_filtered$table[,"low_expression"] <- "F" #####  Low-expression column ##### 
            tt_filtered$table[,"FDR_plot"] <- -log10(tt_filtered$table$FDR) #####  For plotting Volcano, here we use FDR, not pValue  ##### 
            
            
            tt_merged <- merge(tt$table[,c("Gene", "logFC", "logCPM", 'F', 'PValue', 'FDR')], 
                               tt_filtered$table[,c("Gene", "FDR", "low_expression", "FDR_plot")], 
                               by="Gene", all=T)   
            
            # Replace NA FDRs with NA                   #####  ORDER CHANGED #####  
            tt_merged[which(is.na(tt_merged$FDR)),"FDR"] <- 1 
            
            EXPRESSION_MEAN <- genes_tofilter_sub[,c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)]
            colnames(EXPRESSION_MEAN) <- c("Gene", "mean_exp_control", 'mean_exp_genotype')
            tt_merged <- merge(tt_merged, EXPRESSION_MEAN, by="Gene")
    
            
            tt_merged[,"Region"] <- REGION
            tt_merged[,"annotation_level"] <- CELL_LEVEL
            tt_merged[,"cell_state"] <- CELL_STATE
            tt_merged[,"cell_type"] <- CELLTYPE
            
            
            tt_merged[,"comparison"] <- paste("control_", GENOTYPE, sep="")
            
            ###### Suggestion to add those columns
            tt_merged[,"Observations_genotype"] <- as.numeric(table(meta.data_sub[,"Condition_custom"])[GENOTYPE])
            tt_merged[,"Observations_reference"] <- as.numeric(table(meta.data_sub[,"Condition_custom"])['Control'])
            
            # low expression add T
            tt_merged[which(is.na(tt_merged$low_expression)),"low_expression"] <- "T"
            
            if(i==1){
                 final_df <- tt_merged   
            } else {
                final_df <- rbind(final_df, tt_merged)
            }
            
            i <- i + 1
            message("\n###FINISHED: ", CELL_STATE," #####\n")
        } else {
            message("\n###Skip: ", CELL_STATE," #####\n")
        }
    }
}

In [ ]:
colnames(final_df) <- gsub("FDR.x", "FDR_all", colnames(final_df))
colnames(final_df) <- gsub("FDR.y", "FDR_onlyhigh", colnames(final_df))

In [43]:
#write.csv(final_df, "/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_CD4CD8NK_ALLGENOTYPES_EDGER_LV_V4_NOTTM_CUTOFF30.csv")